# Doc2Vec Lee Dataset

In [1]:
# Importando as bibliotecas que vamos utilizar
import gensim
import os
import collections
import smart_open
import random

## O Dataset Lee Corpus

* [Lee Corpus](http://faculty.sites.uci.edu/mdlee/similarity-data/)

## Iniciando

Para começar, precisamos ter um conjunto de documentos para treinar nosso modelo doc2vec. Em teoria, um documento poderia ser qualquer coisa, desde um tweet curto de 140 caracteres, um único parágrafo (ou seja, um resumo de um artigo de revista), um artigo de notícias ou um livro. No jargão de PNL, uma coleção ou conjunto de documentos é geralmente chamado de um <b> corpus</b>.

In [2]:
# Conjunto de arquivos para treino e teste
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

## Definição de uma função para ler e processar o texto

Abaixo, definimos uma função para abrir o arquivo de treino/teste (com codificação latina), ler o arquivo linha por linha, pré-processar cada linha usando uma ferramenta simples de pré-processamento gensim (por exemplo, tokenizar texto em palavras individuais, remover a pontuação, alterar o texto para minúsculos, etc.) e retorna uma lista de palavras. Para treinar o modelo, precisamos associar uma tag a cada documento do corpus de treino. No nosso caso, a tag é simplesmente o número da linha com base em zero.

In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [4]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

/home/geanderson/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Vamos verificar o conjunto de treino

In [5]:
train_corpus

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

Vamos verificar o conjunto de treino

In [6]:
test_corpus

[['the',
  'national',
  'executive',
  'of',
  'the',
  'strife',
  'torn',
  'democrats',
  'last',
  'night',
  'appointed',
  'little',
  'known',
  'west',
  'australian',
  'senator',
  'brian',
  'greig',
  'as',
  'interim',
  'leader',
  'shock',
  'move',
  'likely',
  'to',
  'provoke',
  'further',
  'conflict',
  'between',
  'the',
  'party',
  'senators',
  'and',
  'its',
  'organisation',
  'in',
  'move',
  'to',
  'reassert',
  'control',
  'over',
  'the',
  'party',
  'seven',
  'senators',
  'the',
  'national',
  'executive',
  'last',
  'night',
  'rejected',
  'aden',
  'ridgeway',
  'bid',
  'to',
  'become',
  'interim',
  'leader',
  'in',
  'favour',
  'of',
  'senator',
  'greig',
  'supporter',
  'of',
  'deposed',
  'leader',
  'natasha',
  'stott',
  'despoja',
  'and',
  'an',
  'outspoken',
  'gay',
  'rights',
  'activist'],
 ['cash',
  'strapped',
  'financial',
  'services',
  'group',
  'amp',
  'has',
  'shelved',
  'million',
  'plan',
  'to',
 

## Treinamento do Modelo

### Doc2Vec 

Agora, instanciamos um modelo Doc2Vec com um tamanho de vetor com 50 palavras e iterando o corpus de treinamento 40 vezes. Definimos a contagem mínima de palavras como 2 para descartar palavras com muito poucas ocorrências. (Sem uma variedade de exemplos representativos, manter essas palavras pouco frequentes pode piorar o modelo!)

In [7]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

### Construindo o Vocabulário

In [8]:
model.build_vocab(train_corpus)

Essencialmente, o vocabulário é um dicionário (acessível via `model.wv.vocab`) de todas as palavras únicas extraídas do corpus de treinamento junto com a contagem (por exemplo,` model.wv.vocab['penalty'].count` para contagens da palavra `penalty`).

In [9]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 2.9 s, sys: 52.2 ms, total: 2.95 s
Wall time: 1.19 s


### Inferindo os Vetores

Um detalhe importante a ser observado é que agora você pode inferir um vetor para qualquer parte do texto sem precisar treinar novamente o modelo passando uma lista de palavras para a função `model.infer_vector`. Este vetor pode ser comparado com outros vetores via semelhança de cosseno.

In [10]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([ 0.01044543, -0.13288416, -0.1423525 ,  0.1525093 , -0.02168774,
        0.00434739, -0.09646566,  0.0066804 ,  0.24499352,  0.01894794,
       -0.38358557, -0.06204506, -0.06812683, -0.1057732 ,  0.1553441 ,
       -0.04901268, -0.2611716 ,  0.20173861,  0.01142475,  0.07505932,
       -0.10948537, -0.09878116,  0.1081218 ,  0.00320669,  0.1731693 ,
       -0.13993186, -0.04629424, -0.13266541,  0.0095556 , -0.00071516,
        0.0164416 , -0.24608105, -0.03251071,  0.03551777,  0.03092943,
        0.20602207, -0.00776759,  0.0733397 , -0.17529339, -0.20040998,
        0.14124897, -0.00468647,  0.15965307,  0.05859555,  0.065355  ,
        0.1905701 , -0.24130844, -0.14424777,  0.08869192,  0.12689969],
      dtype=float32)

Note que `infer_vector()` * não * aceita uma string, mas sim uma lista de tokens de string, que já deveriam ter sido tokenizados da mesma maneira que a propriedade `words` dos objetos originais do documento de treinamento.

Observe também que, como os algoritmos de treinamento / inferência subjacentes são um problema de aproximação iterativo que utiliza a randomização interna, inferências repetidas do mesmo texto retornarão vetores ligeiramente diferentes.

## Modelo de Avaliação

Para avaliar nosso novo modelo, primeiro inferimos novos vetores para cada documento do corpus de treinamento, comparamos os vetores inferidos com o corpus de treinamento e, em seguida, retornamos a classificação do documento com base na auto-similaridade. Basicamente, estamos fingindo que o corpus de treinamento é um dado novo e invisível e depois vemos como eles se comparam ao modelo treinado. A expectativa é que provavelmente exageramos no modelo (ou seja, todos os níveis serão inferiores a 2) e, portanto, poderemos encontrar documentos semelhantes com muita facilidade. Além disso, acompanharemos as segundas fileiras para uma comparação de documentos menos semelhantes.

In [11]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

Vamos contar como cada documento é classificado em relação ao corpus de treinamento

In [12]:
collections.Counter(ranks)

Counter({0: 292, 1: 8})

Vamos contar como cada documento é classificado em relação ao treinamento. Basicamente, mais de 95% dos documentos inferidos são mais similares a si mesmo e cerca de 5% do tempo é erroneamente mais semelhante a outro documento. a verificação de um vetor inferido contra um vetor de treinamento é uma espécie de "verificação de sanidade" para determinar se o modelo está se comportando de maneira útil consistente, embora não seja um valor real de "precisão".

Isso é ótimo e não é totalmente surpreendente. Podemos dar uma olhada em um exemplo:

In [13]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Observe acima que o documento mais semelhante (geralmente o mesmo texto) possui uma pontuação de similaridade próxima de 1.0. No entanto, a pontuação de similaridade para os documentos de segunda classificação deve ser significativamente menor (supondo que os documentos sejam de fato diferentes) e o raciocínio se torna óbvio quando examinamos o próprio texto.

Podemos executar a próxima célula repetidamente para ver uma amostra de outras comparações de documentos de destino.

In [14]:
# Pegue um documento aleatorio do corpus e infere o vetor do modelo
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare e impriva o segundo documento mais similar
print('Documento de Treino ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Documento {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Documento de Treino (91): «the flanders graveyard of thousands of australian world war soldiers in belgium could be overrun by motorway flemish authorities are considering new bypass through the heart of the pilkem ridge battlefield the site of the opening infantry campaign of the third battle of ypres from july to november in which australian servicemen lost their lives five thousand of the australians are accounted for but if the proposed route gets the go ahead many of the thousands still missing will be buried under bitumen and heavy traffic the proposed route would split the battlefield in two and also run close to about dozen war cemeteries in belgium maintained by the commonwealth war graves commission the route of the motorway is to be decided early next year»

Similar Documento (178, 0.727117657661438): «year old middle eastern woman is said to be responding well to treatment after being diagnosed with typhoid in temporary holding centre on remote christmas island it could be 

## Testando o Modelo

Usando a mesma abordagem acima, inferiremos o vetor para um documento de teste escolhido aleatoriamente e comparamos o documento com o nosso modelo a olho nu

In [15]:
# Pegue um documento aleatorio do corpus e infere o vetor do modelo
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare e imprima o mais similar/médio/menos similar do conjunto de treino
print('Documento de Teste ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS POR MODELO %s:\n' % model)
for label, index in [('MAIS SIMILAR', 0), ('MEDIO', len(sims)//2), ('MENOS SIMILAR', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Documento de Teste (40): «researchers conducting the most elaborate wild goose chase in history are digesting the news that bird they have tracked for over miles is about to be cooked kerry an irish light bellied brent goose was one of six birds tagged in northern ireland in may by researchers monitoring the species remarkable migration last week however he was found dead in an inuit hunter freezer in canada still wearing his satelite tracking device kerry was discovered by researchers on the remote cornwallis island they picked up the signal and decided to try to find him»

SIMILAR/DISSIMILAR DOCS POR MODELO Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MAIS SIMILAR (188, 0.7918196320533752): «one person has died after royal flying doctor service rfds aircraft crashed near the city of mt gambier in south australia south east last night the rfds says beech aircraft apparently came down just before midnight acdt in an area called dismal swamp about kilometres north of mt gambier the aircraft 

### Finalizando